In [231]:
from gym_2048.envs.game2048_env import Game2048Env
import game2048_env
import gym
import numpy as np
import copy
import pandas as pd
from collections import defaultdict
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
ENV_NAME = "2048-v0"
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

[123]

In [158]:

np.random.seed(123)
env.seed(22)
env.reset()
print(env.Matrix)
print(get_legal_move(env))
env.step(3)
print(env.Matrix)

[[2 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[0, 1, 2]
[[2 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [216]:
score_10 = []
tile_10 = []
score_100 = []
tile_100 = []
score_500 = []
tile_500 = []


In [225]:
env.seed(123)
env.reset()
for i in range(10):
    score, tile = better_monte_carlo_new_random(env, 10, 20, prints=False)
    score_10.append(score)
    tile_10.append(tile)
    env.reset()

In [226]:
env.seed(123)
env.reset()
for i in range(5):
    score, tile = better_monte_carlo_new_random(env, 100,0, prints=False)
    score_100.append(score)
    tile_100.append(tile)
    env.reset()

In [271]:
tile_10[0:5]

[2048, 1024, 1024, 2048, 1024]

In [290]:
a = pd.MultiIndex.from_tuples([("10_random_games_per_move", "best_tile"), 
                         ("10_random_games_per_move", "score"),
                          ("100_random_games_per_move", "best_tile"),
                          ("100_random_games_per_move", "score")])
df = pd.DataFrame({"best_tile_10": tile_10[0:5], "score_10": score_10[0:5], "best_tile_100": tile_100,  "score_100":score_100})
df.columns = a
df.index = ["game 0", "game 1", "game 2", "game 3","game 4"]
df

10_random_games_per_move          100_random_games_per_move         
                      best_tile    score                 best_tile    score
game 0                     2048  27508.0                      2048  36604.0
game 1                     1024  12404.0                      4096  60632.0
game 2                     1024  14572.0                      4096  60700.0
game 3                     2048  32308.0                      2048  36216.0
game 4                     1024  16164.0                      2048  34592.0

In [224]:
def better_monte_carlo_new_random(game, number_of_games, depth, prints = False):
    
    def get_legal_moves(game):
        legal_moves = []
        temp_game =copy.deepcopy(game)
        Matrix = copy.deepcopy(game.Matrix)
        for i in range(4):
            temp_game.step(i)
            if not np.array_equal(temp_game.Matrix, Matrix):
                legal_moves.append(i)
                temp_game.Matrix = copy.deepcopy(Matrix)
        return legal_moves
            
    def random_game_initial_score(game, depth, initial_move):
        counter =0
        _,_, lost, _ = game.step(initial_move)
        while ( not lost):
             _, _, lost, _ = game.step(np.random.randint(4))
        return initial_move, game.score
    def getmove(game, number_of_games):
        grids = []
        possible_moves = get_legal_moves(game)
#        print(possible_moves)
        for i in range(number_of_games):
            grids.append(copy.deepcopy(game))
        move_scores = defaultdict(list)
        i=0
        for games in grids:
            initial_move, score = random_game_initial_score(games, depth, possible_moves[i])
            move_scores[initial_move].append( score)
            i+=1
            i=i%(len(possible_moves))
        score = 0
        for key, value in move_scores.items():
            if (prints == True):
                print(key,sum(value)/len(value))
            if (sum(value)/len(value) >score ):
                    move = key
                    score = sum(value)/len(value)
        if (prints == True):
            print(move, score)
        return move
            

    states = []
    lost = False
    while (not lost):
        move = getmove(game, number_of_games)
        if (prints == True):
            print(game.Matrix)
        _, _, lost, _ = game.step(move)
    return(game.score, game.highest())
    #return states

In [215]:
env.highest()

2048